Importing the libraries

In [168]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

1 represent fake news and
0 represent real news

In [140]:
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [141]:
#printing the stopwords in english
st_words=stopwords.words("english")
print(st_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing


In [142]:
#loading the dataset into pandas dataframe
news_dataset=pd.read_csv("train.csv")

In [143]:
news_dataset.shape

(20800, 5)

In [144]:
#printing the first five row
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [145]:
#checking for missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [146]:
#replacing the null value
news_dataset=news_dataset.fillna("")

In [147]:
#merging the author name and news title
news_dataset["content"]=news_dataset["author"] + " " + news_dataset["title"]

In [148]:
news_dataset["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [149]:
#separating the data and label
features=news_dataset.drop("label",axis=1)
target=news_dataset["label"]

In [150]:
features.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


Stemming

Stemming is the process of reducing a word to it root word

In [151]:
port_stem=PorterStemmer()

In [152]:
def stemming(content):
    stemmed_content=re.sub("[^a-zA-Z]"," ",content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not in st_words]
    stemmed_content=" ".join(stemmed_content)
    return stemmed_content 

In [153]:
news_dataset["content"]=news_dataset["content"].apply(stemming)

In [154]:
news_dataset["content"].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [155]:
#new feature
feature=news_dataset["content"].values

In [156]:
#converting textual data to numerical data
vector=TfidfVectorizer()
vector.fit(feature)
feature=vector.transform(feature)


In [157]:
#spliting the data into train and test data
x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2,stratify=target,random_state=2)

In [158]:
#checking if the data is balanced
y_train.value_counts()

1    8330
0    8310
Name: label, dtype: int64

Training the model

In [161]:
#instantiating and fiting the model
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

Model Evaluation

In [163]:
#prediction on training data
train_pred=model.predict(x_train)
training_data_accuracy=accuracy_score(train_pred,y_train)
print(training_data_accuracy)

0.9865985576923076


In [164]:
#prediction on test data
test_pred=model.predict(x_test)
test_data_accuracy=accuracy_score(test_pred,y_test)
print(test_data_accuracy)

0.9790865384615385
